In [52]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

app.config["SQLALCHEMY_DATABASE_URI"] = "mssql+pyodbc://LAPTOP-Q0DB1ITI\SQLEXPRESS/DB?driver=SQL+Server?trusted_connection=yes"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
db = SQLAlchemy(app)

class audio_data(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(500), nullable=False)
    audio_path = db.Column(db.String(500), nullable=False)
    
    def __repr__(self):
        return "<Content %r>" % self.content

In [53]:
db.create_all()

In [1]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

# url definition
url = "https://www.theguardian.com/uk"

# Request
r1 = requests.get(url)
r1.status_code

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification
coverpage_news = soup1.find_all('h3', class_='fc-item__title')

number_of_articles = len(coverpage_news)

# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

for n in tqdm(np.arange(0, number_of_articles)):
        
        # We need to ignore "live" pages since they are not articles
        if "live" in coverpage_news[n].find('a')['href']:  
                continue

        # Getting the link of the article
        link = coverpage_news[n].find('a')['href']
        list_links.append(link)

        # Getting the title
        title = coverpage_news[n].find('a').get_text()
        list_titles.append(title)

        # Reading the content (it is divided in paragraphs)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        #print(soup_article)
        body = soup_article.find_all('p', class_='dcr-s23rjr')

        # Unifying the paragraphs
        list_paragraphs = []
        for p in body:
                paragraph = p.get_text()
                list_paragraphs.append(paragraph)
                final_article = " ".join(list_paragraphs)

        news_contents.append(final_article)
        
all_content = []
for news in news_contents:
        tmp_content = news.replace('“', '').replace('”','').replace('’','').split(". ")
        
        for tmp in tmp_content:
                for tmp2 in tmp.split(", "):
                        all_content.append(tmp2)

final_content = []
statistic = []
for unique_content in np.unique(all_content):
    if len(unique_content.strip()) >= 500:
        final_content.append(unique_content.strip()[:499])
    else:
        final_content.append(unique_content.strip())
    statistic.append(len(unique_content.strip()))

100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [02:36<00:00,  1.30s/it]


In [5]:
from app import TextAudioPair
from app import db

In [6]:
for index, content in tqdm(enumerate(final_content[1:])):
        tmp_audio_pair = TextAudioPair(text=f"{content}")
        db.session.add(tmp_audio_pair)
        db.session.commit()

2596it [00:06, 413.66it/s]


In [56]:
audio_data.query.all()

[<Content 'Additional reporting Fiona Harvey'>,
 <Content 'But under huge pressure and after an adverse court ruling that raised the threat of heavy fines'>,
 <Content 'Failure to pass the legislation would be disastrous for the US and the global community'>,
 <Content 'GW Pharmaceuticals GW was acquired by the US firm Jazz Pharmaceuticals for $7.2bn in February'>,
 <Content 'Ione Belarra'>,
 <Content 'Simmering resentment at the president exploded outside the White House last week'>,
 <Content 'The French president'>,
 <Content 'The bill includes a program of payments and penalties to ensure utilities phase out fossil fuels from Americas electricity supply'>,
 <Content 'There are over 9% more radiology doctors compared to the same period in 2019 and we have provided ￡52m to further invest in the cancer and diagnostics workforce over the next two years.'>,
 <Content 'Tom Dunning'>,
 <Content 'Were continuing to support businesses including through the super deduction – the biggest two-